In [1]:
import numpy as np

import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

TRAIN_COUNT = 60
VALIDATION_COUNT = 20
TEST_COUNT = 20

Using cuda device


In [2]:
from models.UNet import *
from datasets.NeurosismDataset import *
import torchvision.transforms as tf
from torch.utils.data import DataLoader

dataPath = "../../sensorium_data/"
mouseId = "dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce"

overlayCount = 20
trainData = NeurosismDataset(overlayCount, dataPath + mouseId, (122, 2, 2), densify=False, seed=42)

batchSize = 1
shuffle = True
numWorkers = 4
trainLoader = DataLoader(trainData, batch_size=batchSize, shuffle=shuffle, num_workers=numWorkers)

model = UNet().to(device)
# y = model(x)

In [11]:
responseStds = np.load(dataPath + mouseId + "/meta/statistics/responses/all/std.npy")
responseStds.shape

(7440, 324)

In [18]:
responses = np.load(dataPath + mouseId + "/data/responses/0.npy")
I, J = responses.shape
nans = 0
for i in range(I):
    for j in range(J):
        if math.isnan(responses[i][j]):
            nans += 1
nans

178560

In [16]:
responses.shape

(7440, 324)

In [3]:
# Step 3: Loss Function
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()

In [4]:
# Step 4: Optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.01)
# optimizer = optim.RMSprop(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [14]:
import math
math.isnan(trainData.threshold)

True

In [19]:
trainData.threshold > 0

False

In [5]:
# Step 5: Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for x, y in trainLoader:
        x = x.cuda()
        y = y.cuda()
        yPred = model(x)
        loss = criterion(yPred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        print(loss)

    # Print average loss per epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainLoader)}")

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(nan, device='cuda:0', gr

KeyboardInterrupt: 

In [ ]:
# Step 6: Validation/Test
model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:  # Assuming val_loader is DataLoader
        outputs = model(inputs)
        # Evaluate model performance (e.g., compute accuracy, loss, etc.)